#VideoClassification

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2 as cv2
import random as random
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import datasets, layers, models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


##Reading Data

In [ ]:
os.chdir("drive/My Drive/")

In [ ]:
!ls

 Book2and3.pdf		    final_project.doc	        usethis
'Colab Notebooks'	   'shaykh shoaib'
'final_project copy.docx'   SportsVideoClassification


In [ ]:
os.chdir('SportsVideoClassification/Data')

In [ ]:
def read_data(train_path = "train/",test_path = "test/" ):
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale = 1./255,
      validation_split=0.2,
      shear_range = 0.2, # random application of shearing
      zoom_range = 0.2,
      horizontal_flip = True) # randomly flipping half of the images horizontally

  train_generator = train_datagen.flow_from_directory(
      train_path,
      target_size = (256, 256),
      shuffle = True,
      color_mode = "rgb")
  test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
  validation_generator = test_datagen.flow_from_directory(
      test_path,
      target_size = (256, 256),
      batch_size = 15,
      shuffle=True,
      color_mode = 'rgb')
  return (train_generator,validation_generator)

train_generator,validation_generator = read_data("data3/train/","data3/test/")


Found 3660 images belonging to 3 classes.
Found 1590 images belonging to 3 classes.


##**Modelling**

###Part 1: Spacial Featuring
---

#### Modelling

In [ ]:
model1 = models.Sequential()
model1.add(layers.Conv2D(32, (3,3), input_shape = (256, 256, 3), activation = 'relu'))
model1.add(layers.MaxPooling2D(pool_size = (2,2)))
model1.add(layers.Dropout(0.05))
model1.add(layers.Conv2D(64, (3,3), activation = 'relu'))
model1.add(layers.MaxPooling2D(pool_size = (2,2)))
model1.add(layers.Dropout(0.35))
model1.add(layers.Flatten())
model1.add(layers.Dense(200,activation = 'relu'))
model1.add(layers.Dense(3, activation = 'softmax'))
model1.compile(loss = tf.keras.losses.CategoricalCrossentropy(),optimizer= 'adam',
              metrics = ['accuracy'])

In [ ]:
model1.load_weights("modelCNN.h5")

ValueError: ignored

In [ ]:
#model1.fit(train_generator,validation_data=validation_generator,epochs = 10,batch_size = 16)
model1.save_weights("modelCNN.h5")

In [ ]:
model1.evaluate(validation_generator)

106/106 [==============================] - 44s 413ms/step - loss: 1.9096 - accuracy: 0.5975


[1.9096027612686157, 0.597484290599823]

In [ ]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 62, 62, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 246016)           

####Feature Extraction
---
There two options to extract layers from


1.   Last Dense Layer
2.   Last Convoluted layer
3.   Last layer from InceptionResNetV2

Convoluted layer is found to reach maximum of 50% accuracy on LSTM model in video prediction, while last Dense layer is found to have lower accuracy that is lesser than 50% on evaluation data with different number of units.Also, This may well change with change in number of nodes of dense layer and with change in data. InceptionRestNetV2 on imagenet trained wts. had much lesser accuracy of 20%. Other available models also did not perform any better.



#####Features: Dense layer 

In [ ]:
modelfeatured1 = models.Model(
    inputs=model1.input,
    outputs=model1.get_layer('dense_2').output
)

#####Features: Convoluted layer

In [ ]:
modelfeatured2 = models.Model(
    inputs=model1.input,
    outputs=model1.get_layer('flatten').output
)

In [ ]:
base_model = tf.keras.applications.inception_v3.InceptionV3(
    weights='imagenet',
    include_top=True
)
# We'll extract features at the final pool layer.
modelfeatured3 = models.Model(
    inputs=base_model.input,
    outputs=base_model.get_layer('avg_pool').output
)

In [ ]:
def read_data(train_path = "train/",test_path = "test/" ):
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale = 1./255,
      #validation_split=0.2,
      shear_range = 0.2, # random application of shearing
      zoom_range = 0.2,
      horizontal_flip = True) # randomly flipping half of the images horizontally

  train_generator = train_datagen.flow_from_directory(
      train_path,
      target_size = (256, 256),
      shuffle = False,
      color_mode = "rgb")
  test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
  validation_generator = test_datagen.flow_from_directory(
      test_path,
      target_size = (256, 256),
      batch_size = 15,
      shuffle=False,
      color_mode = 'rgb')
  return (train_generator,validation_generator)

train_generator2,validation_generator2 = read_data("data3/train/","data3/test/")

Found 3660 images belonging to 3 classes.
Found 1590 images belonging to 3 classes.


In [ ]:
'''modelfeatured as on your feature model from the above'''

modelfeatured = modelfeatured1
#modelfeatured = modelfeatured2
#modelfeatured = modelfeatured3

train_featured = modelfeatured.predict(train_generator)
test_featured = modelfeatured.predict(validation_generator)

In [ ]:
np.savetxt('data3/ftrainf.csv',train_featured, delimiter=',')
np.savetxt('data3/ftestf.csv',test_featured, delimiter=',')

### Part 2: Temporal Featuring

In [ ]:
!ls

data3  Data4  dataset  features  modelCNN.h5  modelFF.h5  modelLstm.h5


In [ ]:
DATADIR="Data4/usethis/final/multiribzv5/"

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale = 1./255,
      shear_range = 0.2, # random application of shearing
      zoom_range = 0.2,
      validation_split=0.2,
      horizontal_flip = True) # randomly flipping half of the images horizontally

train_generator = train_datagen.flow_from_directory(
      DATADIR,
      target_size = (256, 256),
      shuffle = True,
      color_mode = "rgb")

Found 1079 images belonging to 4 classes.


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(256, kernel_size=(1, 1),input_shape=(256,256, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(layers.MaxPooling2D((3, 3)))

model.add(layers.Conv2D(128, (3, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(3, 3)))

model.add(layers.Conv2D(64, (3, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (1, 1)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.3))

model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.3))       
model.add(layers.Dense(4, activation='softmax'))

In [ ]:
model.compile(optimizer="adam",loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])
model.load_weights('modelFF.h5')
#callback = tf.keras.callbacks.EarlyStopping(patience=3)
#model.fit(train_generator,epochs=9,batch_size=6)
#model.save_weights("modelFF.h5")

In [ ]:
model.evaluate_generator(train_generator)

In [ ]:
train_datagen =  tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'data3/train',
        target_size=(256, 256),
        color_mode="rgb",
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

filenames = train_generator.filenames
nb_samples = len(filenames)

predict = model.predict_generator(train_generator)
frame = pd.DataFrame(predict, columns=[1,2,3,4])
feature_train_1 = pd.DataFrame(frame.idxmax(axis=1),columns=["cat"])

In [ ]:
test_datagen1 =  tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator1 = test_datagen1.flow_from_directory(
        'data3/test',
        target_size=(256, 256),
        color_mode="rgb",
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

#filenames = test_generator1.filenames
#nb_samples = len(filenames)

#predict = model.predict_generator(test_generator1)
#frame = pd.DataFrame(predict, columns=[1,2,3,4])
#feature_test_1 = pd.DataFrame(frame.idxmax(axis=1),columns=["cat"])

Found 1590 images belonging to 3 classes.


In [ ]:
#os.makedirs("features")

In [ ]:
feature_test_1

In [ ]:
#feature_test_1.to_csv('features/feature_test_1.csv', index=False)
#feature_train_1.to_csv('features/feature_train_1.csv', index=False)

In [ ]:
feature_test_1 = pd.read_csv('features/feature_test_1.csv')
feature_train_1 = pd.read_csv('features/feature_train_1.csv')

In [ ]:
feature_test_1[340:400]

###Part 3: LSTM Modelling

-----
Here LSTM model is build on the sequenced dataframes from each video
These dataframes have

 ***extracted***

1.   *spacial features from the cnn model*
2.   *temporal features from cnn or other models*

timesteps depend on the set frame rate



In [ ]:
df = np.genfromtxt('data3/ftrainf.csv', delimiter=',')
scaler=MinMaxScaler()

scaled=scaler.fit_transform(df)
df=pd.DataFrame(scaled)

kmean4=KMeans(n_clusters=4)
kmean4.fit(df)
y_hat = kmean4.predict(df)
feature_train_2 = y_hat.reshape(y_hat.shape[0],1)


In [ ]:
df = np.genfromtxt('data3/ftestf.csv', delimiter=',')
scaler=MinMaxScaler()

scaled=scaler.fit_transform(df)
df=pd.DataFrame(scaled)

kmean4=KMeans(n_clusters=4)
kmean4.fit(df)
y_hat = kmean4.predict(df)
feature_test_2 = y_hat.reshape(y_hat.shape[0],1)

In [ ]:
#np.savetxt('features/feature_test_2.csv',feature_test_2, delimiter=',')
#np.savetxt('features/feature_train_2.csv',feature_train_2, delimiter=',')
feature_test_2 = np.genfromtxt('features/feature_test_2.csv', delimiter=',')
feature_train_2 = np.genfromtxt('features/feature_train_2.csv', delimiter=',')
feature_train_2 = feature_train_2.reshape(feature_train_2.shape[0],1)
feature_test_2 = feature_test_2.reshape(feature_test_2.shape[0],1)

In [ ]:

train_featured = np.genfromtxt('data3/ftrainf.csv', delimiter=',')
test_featured = np.genfromtxt('data3/ftestf.csv', delimiter=',')
n_steps = 30  ##We set frame rate as 30
n_features = test_featured.shape[1]+2

In [ ]:
test_featured = np.append(test_featured,feature_test_1,axis=1)
train_featured = np.append(train_featured,feature_train_1,axis=1)
test_featured = np.append(test_featured,feature_test_2,axis=1)
train_featured = np.append(train_featured,feature_train_2,axis=1)

####Data Processing

In [ ]:
train_featured.shape

(3660, 202)

In [ ]:

temp = train_featured[:n_steps]
temp = temp.reshape(1,n_steps,temp.shape[1])
for i in range(n_steps,train_featured.shape[0],n_steps):
  temp = np.append(temp,train_featured[i:i+n_steps].reshape(1,n_steps,train_featured.shape[1]),axis = 0)
trainX = temp
temp = test_featured[:n_steps]
temp = temp.reshape(1,temp.shape[0],temp.shape[1])
for i in range(n_steps,test_featured.shape[0],n_steps):
  temp = np.append(temp,test_featured[i:i+n_steps].reshape(1,n_steps,test_featured.shape[1]),axis = 0)
testX = temp

In [ ]:
trainX.shape

(122, 30, 202)

In [ ]:

train_generator,validation_generator = read_data("data3/train/","data3/test/")

Found 3660 images belonging to 3 classes.
Found 1590 images belonging to 3 classes.


In [ ]:
y_train = []
for i in train_generator.filenames:
  if(i.split("/")[0]=="CricketBowling"):
    y_train.append(-100) 
  elif(i.split("/")[0]=="HighJump"):
    y_train.append(0)
  elif(i.split("/")[0]=="LongJump"):
    y_train.append(100) 

y_test = []
for i in validation_generator.filenames:
  if(i.split("/")[0]=="CricketBowling"):
    y_test.append(-100) 
  elif(i.split("/")[0]=="HighJump"):
    y_test.append(0)
  elif(i.split("/")[0]=="LongJump"):
    y_test.append(100) 

trainY=[]
for i in range(0,len(y_train),n_steps):
  trainY.append(y_train[i])
testY=[]
for i in range(0,len(y_test),n_steps):
  testY.append(y_test[i])


trainY = np.array(trainY)
testY = np.array(testY)

trainY=[]
for i in range(0,len(y_train),n_steps):
  trainY.append(y_train[i])
testY=[]
for i in range(0,len(y_test),n_steps):
  testY.append(y_test[i])



TRAINy = pd.DataFrame(trainY)
TRAINy.columns = ['label']
TRAINy['1']=0
TRAINy['2']=0
TRAINy['3']=0
for i in TRAINy.loc[TRAINy['label']==-100].index:
  TRAINy.loc[i,"1"]=1
for i in TRAINy.loc[TRAINy['label']==0].index:
  TRAINy.loc[i,"2"]=1
for i in TRAINy.loc[TRAINy['label']==100].index:
  TRAINy.loc[i,"3"]=1
final_trainY =np.array(TRAINy[['1','2','3']])

TESTy = pd.DataFrame(testY)
TESTy.columns = ['label']
TESTy['1']=0
TESTy['2']=0
TESTy['3']=0
for i in TESTy.loc[TESTy['label']==-100].index:
  TESTy.loc[i,"1"]=1
for i in TESTy.loc[TESTy['label']==0].index:
  TESTy.loc[i,"2"]=1
for i in TESTy.loc[TESTy['label']==100].index:
  TESTy.loc[i,"3"]=1
final_testY = np.array(TESTy[['1','2','3']])


#### Model Building

In [ ]:
testX.shape

(53, 30, 202)

In [ ]:
model2 = models.Sequential()
model2.add(layers.LSTM(500, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model2.add(tf.keras.layers.BatchNormalization())
model2.add(layers.LSTM(500, activation='relu'))
model2.add(layers.Dense(3,activation="softmax"))


In [ ]:
final_trainY.shape

(122, 3)

In [ ]:

#optimizer = tf.keras.optimizers.Adam(lr=1e-4, decay=1e-5)
model2.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer='adam',
                    metrics=['accuracy','top_k_categorical_accuracy'])
#model2.load_weights("modelLstm.h5")
model2.fit(trainX,final_trainY,validation_data=(testX,final_testY),batch_size=8,epochs=20)
#model2.evaluate(testX,final_testY,batch_size = 8)
#model2.save_weights("modelLstm.h5")

Epoch 1/20
16/16 [==============================] - 7s 455ms/step - loss: 1.0688 - accuracy: 0.4672 - top_k_categorical_accuracy: 1.0000 - val_loss: 5.4792 - val_accuracy: 0.4717 - val_top_k_categorical_accuracy: 1.0000
Epoch 2/20
16/16 [==============================] - 7s 421ms/step - loss: 1.1059 - accuracy: 0.5410 - top_k_categorical_accuracy: 1.0000 - val_loss: 7.2147 - val_accuracy: 0.4717 - val_top_k_categorical_accuracy: 1.0000
Epoch 3/20
16/16 [==============================] - 7s 428ms/step - loss: 0.9325 - accuracy: 0.5656 - top_k_categorical_accuracy: 1.0000 - val_loss: 719864.0000 - val_accuracy: 0.4717 - val_top_k_categorical_accuracy: 1.0000
Epoch 4/20
16/16 [==============================] - 7s 428ms/step - loss: 0.8964 - accuracy: 0.5656 - top_k_categorical_accuracy: 1.0000 - val_loss: 2538.2471 - val_accuracy: 0.3585 - val_top_k_categorical_accuracy: 1.0000
Epoch 5/20
16/16 [==============================] - 7s 424ms/step - loss: 0.7927 - accuracy: 0.6803 - top_k_cate

In [ ]:
model2.evaluate(testX)

2/2 [==============================] - 0s 57ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - top_k_categorical_accuracy: 0.0000e+00


[0.0, 0.0, 0.0]

In [ ]:
model2.save_weights("modelLstm.h5")

##Predictions

In [ ]:
predicts = model2.predict(trainX)
frame = pd.DataFrame(predicts, columns=[1,2,3])
classification = pd.DataFrame(frame.idxmax(axis=1),columns=["cat"])

In [ ]:
classification

,cat
0,1
1,1
2,1
3,2
4,1
...,...
117,1
118,1
119,3
120,2
